In [1]:
import ray
from ray.util import state

In [2]:
ray.init()

2025-11-12 11:56:52,907	INFO worker.py:1771 -- Connecting to existing Ray cluster at address: 172.19.0.4:6379...
2025-11-12 11:56:52,922	INFO worker.py:1942 -- Connected to Ray cluster. View the dashboard at 172.19.0.4:8265 


Python version:,3.10.18
Ray version:,2.49.1
Dashboard:,http://172.19.0.4:8265


In [3]:
# state.list_tasks(filters=[("name", "=", "AnnotationImporter.import_from_tsv_row")], detail=True)[0]

In [4]:
import time

# Define a Ray actor with a function
@ray.remote
class MyActor:
    def compute(self, x):
        return x * x

# Define a Ray remote function that calls a bunch of actors' functions
@ray.remote
def call_actors(values):
    # Call the 'compute' method on each actor with the corresponding value
    actor_handles = [MyActor.remote() for _ in range(len(values))]
    futures = [actor.compute.remote(val) for actor, val in zip(actor_handles, values)]
    return ray.get(futures)


In [5]:

# Call the remote function and store the handle

call_handle = call_actors.remote([1, 2, 3])


In [12]:
limit = 1

# Get the task id of the remote function
task = state.get_task(call_handle)
if task:
    parent_task_id = task.task_id
    # Use parent_task_id to get child tasks
    child_tasks = state.list_tasks(filters=[("type", "=", "NORMAL_TASK")], limit=limit, detail=True)
    for t in child_tasks:
        print(f"Child task {t.name} (id={t.task_id}) state: {t.state}")
else:
    print("No call_actors tasks found.")


Child task call_actors (id=32d950ec0ccf9d2affffffffffffffffffffffff01000000) state: FINISHED


In [9]:
state.list_tasks()

[TaskState(task_id='06f28617326374dba94e6d9eb937b9bec803185701000000', attempt_number=0, name='MyActor.compute', state='FINISHED', job_id='01000000', actor_id='a94e6d9eb937b9bec803185701000000', type='ACTOR_TASK', func_or_class_name='MyActor.compute', parent_task_id='32d950ec0ccf9d2affffffffffffffffffffffff01000000', node_id='2d73f1cf6c50b0a0a821c04acd4678981170eb88e3bb64be188daf9c', worker_id='2f60c7f7f7183b6d2a5042b745e0f3570e98fad928e1863c0b7db643', worker_pid=2739, error_type=None, language=None, required_resources=None, runtime_env_info=None, placement_group_id=None, events=None, profiling_data=None, creation_time_ms=None, start_time_ms=None, end_time_ms=None, task_log_info=None, error_message=None, is_debugger_paused=None, call_site=None, label_selector=None),
 TaskState(task_id='32d950ec0ccf9d2affffffffffffffffffffffff01000000', attempt_number=0, name='call_actors', state='FINISHED', job_id='01000000', actor_id=None, type='NORMAL_TASK', func_or_class_name='call_actors', parent_t